## DEEP LEARNING

# Training LSTM and a simple dense network on the GloVe features 

Word Vectors  

glove vectors - http://www-nlp.stanford.edu/data/glove.840B.300d.zip

In [8]:
from tqdm import tqdm
import numpy as np
import pandas as pd

In [10]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping

Exception ignored in: <bound method tqdm.__del__ of 4204it [01:23, 50.40it/s]>
Traceback (most recent call last):
  File "C:\Users\root\Anaconda3\lib\site-packages\tqdm\_tqdm.py", line 879, in __del__
    self.close()
  File "C:\Users\root\Anaconda3\lib\site-packages\tqdm\_tqdm.py", line 1098, in close
    self._decr_instances(self)
  File "C:\Users\root\Anaconda3\lib\site-packages\tqdm\_tqdm.py", line 438, in _decr_instances
    cls._instances.remove(instance)
  File "C:\Users\root\Anaconda3\lib\_weakrefset.py", line 109, in remove
    self.data.remove(ref(item))
KeyError: <weakref at 0x000001DCB96B2688; to 'tqdm' at 0x000001DCBC6C1128>
Using TensorFlow backend.


In [23]:
# load the GloVe vectors in a dictionary:

embeddings_index = {}
f = open('glove.840B.300d.txt',encoding='utf8')
for line in tqdm(f):
    values = line.split()
    word = values[0]
    try:
        coefs = np.array(values[1:], dtype='float32')
    except ValueError:
        pass
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))


0it [00:00, ?it/s]
705it [00:00, 6445.01it/s]
1639it [00:00, 7491.77it/s]
2671it [00:00, 8139.29it/s]
3714it [00:00, 8488.22it/s]
4743it [00:00, 8672.04it/s]
5744it [00:00, 8751.81it/s]
6813it [00:00, 8897.68it/s]
7891it [00:00, 9017.33it/s]
8959it [00:00, 9100.22it/s]
9888it [00:01, 9039.48it/s]
10842it [00:01, 9010.58it/s]
11910it [00:01, 9073.32it/s]
12959it [00:01, 9113.05it/s]
13994it [00:01, 9137.97it/s]
15055it [00:01, 9175.40it/s]
16129it [00:01, 9215.59it/s]
17180it [00:01, 9238.69it/s]
18226it [00:01, 9256.67it/s]
19218it [00:02, 9246.76it/s]
20256it [00:02, 9258.91it/s]
21318it [00:02, 9280.33it/s]
22363it [00:02, 9292.73it/s]
23358it [00:02, 9284.19it/s]
24299it [00:02, 9201.03it/s]
25183it [00:02, 9104.68it/s]
26014it [00:02, 8998.49it/s]
26848it [00:03, 8942.31it/s]
27748it [00:03, 8917.19it/s]
28535it [00:03, 8731.56it/s]
29246it [00:03, 8659.34it/s]
30151it [00:03, 8647.23it/s]
31197it [00:03, 8675.07it/s]
32240it [00:03, 8700.45it/s]
33293it [00:03, 8727.00it/s]
34314

Found 2195884 word vectors.


In [26]:
test_variants=pd.read_csv('test_variants.csv',sep=",")
training_variants=pd.read_csv('training_variants.csv',sep=",")
t=pd.read_csv('test_text.csv',sep="\|\|", names=["ID","Text"], skiprows=1,header=None)
train=pd.read_csv('training_text.csv',sep="\|\|", names=["ID","Text"],skiprows=1,header=None)
train['Class'] = training_variants['Class']

C:\Users\root\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\root\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.


In [27]:
y = train.Class

In [28]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.Text.values, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [29]:
xtrain = ['-99999' if x is np.nan else x for x in xtrain]
xvalid = ['-99999' if x is np.nan else x for x in xvalid]

In [40]:
# this function creates a normalized vector for the whole sentence
def sent2vec(s):
    words = str(s).lower()#.decode('utf-8')
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

In [38]:
from nltk import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\root\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [44]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\root\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [45]:
# create sentence vectors using the above function for training and validation set
xtrain_glove = [sent2vec(x) for x in tqdm(xtrain)]
xvalid_glove = [sent2vec(x) for x in tqdm(xvalid)]



  0%|          | 0/2988 [00:00<?, ?it/s]

  0%|          | 1/2988 [00:00<27:58,  1.78it/s]

  0%|          | 2/2988 [00:01<41:19,  1.20it/s]

  0%|          | 3/2988 [00:01<31:59,  1.56it/s]

  0%|          | 4/2988 [00:02<25:16,  1.97it/s]

  0%|          | 5/2988 [00:02<21:12,  2.34it/s]

  0%|          | 6/2988 [00:02<20:23,  2.44it/s]

  0%|          | 7/2988 [00:02<18:33,  2.68it/s]

  0%|          | 8/2988 [00:02<17:36,  2.82it/s]

  0%|          | 9/2988 [00:03<16:41,  2.97it/s]

  0%|          | 10/2988 [00:03<15:40,  3.17it/s]

  0%|          | 11/2988 [00:03<14:56,  3.32it/s]

  0%|          | 12/2988 [00:03<14:07,  3.51it/s]

  0%|          | 13/2988 [00:03<13:48,  3.59it/s]

  1%|          | 15/2988 [00:03<12:24,  3.99it/s]

  1%|          | 16/2988 [00:03<11:59,  4.13it/s]

  1%|          | 17/2988 [00:04<12:16,  4.04it/s]

  1%|          | 19/2988 [00:04<11:25,  4.33it/s]

  1%|          | 21/2988 [00:04<11:12,  4.41it/s]

  1%|          | 22/2988 [00:04<10:57,  4.51it/

In [46]:
xtrain_glove = np.array(xtrain_glove)
xvalid_glove = np.array(xvalid_glove)

In [49]:
#imported from kaggle. It is just evaluation metric
def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        print(actual2.shape)
        actual=actual.reshape(actual2.shape[0],1)
        print(actual.shape)
        for i, val in enumerate(actual2.astype(int)):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

In [50]:
import xgboost as xgb

In [51]:
# Fitting a simple xgboost on glove features
clf = xgb.XGBClassifier(nthread=10, silent=False)
clf.fit(xtrain_glove, ytrain)
predictions = clf.predict_proba(xvalid_glove)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

(333, 9)


C:\Users\root\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  # This is added back by InteractiveShellApp.init_path()


(333, 1)
logloss: 2.632 


In [95]:
# scale the data before any neural net:
scl = preprocessing.StandardScaler()
xtrain_glove_scl = scl.fit_transform(xtrain_glove)
xvalid_glove_scl = scl.transform(xvalid_glove)

In [96]:
# we need to binarize the labels for the neural net
ytrain_enc = np_utils.to_categorical(ytrain)
yvalid_enc = np_utils.to_categorical(yvalid)

In [102]:
# create a simple 3 layer sequential neural net
model = Sequential()

model.add(Dense(300, input_dim=300, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(300, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Dense(10))
model.add(Activation('softmax'))

# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [103]:
print (xtrain_glove_scl.shape,ytrain_enc.shape)

(2988, 300) (2988, 10)


In [104]:
model.fit(xtrain_glove_scl, y=ytrain_enc, batch_size=64, 
          epochs=5, verbose=1, 
          validation_data=(xvalid_glove_scl, yvalid_enc))

Train on 2988 samples, validate on 333 samples
Epoch 1/5
2988/2988 [==============================] - 2s 781us/step - loss: 1.8779 - val_loss: 1.4643
Epoch 2/5
2988/2988 [==============================] - 1s 196us/step - loss: 1.3588 - val_loss: 1.2636
Epoch 3/5
2988/2988 [==============================] - 1s 263us/step - loss: 1.1830 - val_loss: 1.1303
Epoch 4/5
2988/2988 [==============================] - 1s 233us/step - loss: 1.0878 - val_loss: 1.0782
Epoch 5/5
2988/2988 [==============================] - 1s 201us/step - loss: 1.0192 - val_loss: 1.0779


In [105]:
predictions= model.predict(xvalid_glove_scl)

In [107]:
print ("logloss: %0.3f " % multiclass_logloss(yvalid_enc, predictions))

logloss: 1.078 
